In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [24]:

input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\input_gwap_luz.csv'
data = pd.read_csv(input_file)

data = data[['GWAP', 'FLOW_LUZ','GWAP_DR','GWAP_FR','GWAP_RD','GWAP_RU']]  # Select the relevant time series column


scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.values)
data_scaler = MinMaxScaler()
columns = ['GWAP', 'FLOW_LUZ','GWAP_DR','GWAP_FR','GWAP_RD','GWAP_RU']
        
X  = data[columns].values
y = data['GWAP'].values.reshape(-1,1)


In [25]:
class TimeSeriesDataset(Dataset):
    
    def __init__(self, X, y, seq_len):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.X) - self.seq_len + 1
        
    def __getitem__(self, idx):
        return (self.X[idx:idx+self.seq_len], self.y[idx+self.seq_len-1])

In [26]:
train_size = int(0.7 * len(data))  # 70% for training
val_size = int(0.15 * len(data))   # 15% for validation
test_size = len(data) - train_size - val_size  # Remaining 15% for testing

train_data = X[:train_size]
train_labels = y[:train_size]

val_data = X[train_size:train_size + val_size]
val_labels = y[train_size:train_size + val_size]

test_data = X[train_size + val_size:]
test_labels = y[train_size + val_size:]
seq_len=1440
batch_size=64

train_dataset = TimeSeriesDataset(train_data, train_labels, seq_len)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=False)

val_dataset = TimeSeriesDataset(val_data, val_labels, seq_len)    
val_dataloader = DataLoader(val_dataset, batch_size, shuffle=False) 

test_dataset = TimeSeriesDataset(test_data, test_labels, seq_len)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)


In [27]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):

    
        # Forward propagate LSTM
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out

In [99]:
class CustomLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.0):
        super(CustomLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        
        # Combined weights for all gates
        self.weight_ih = nn.Parameter(torch.Tensor(4 * hidden_size, input_size))
        self.weight_hh = nn.Parameter(torch.Tensor(4 * hidden_size, hidden_size))
        self.bias = nn.Parameter(torch.Tensor(4 * hidden_size))
        
        self.reset_parameters()
        self.dropout_layer = nn.Dropout(dropout)

    def reset_parameters(self):
        stdv = 1.0 / torch.sqrt(torch.tensor(self.hidden_size, dtype=torch.float))
        for weight in self.parameters():
            nn.init.uniform_(weight, -stdv, stdv)

    def forward(self, input, state):
        hx, cx = state
        hx = self.dropout_layer(hx)  # Apply dropout to the hidden state
        gates = F.linear(input, self.weight_ih, self.bias) + F.linear(hx, self.weight_hh)

        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = F.relu(cellgate)  # Use ReLU here
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * F.relu(cy)  # Use ReLU here as well

        return hy, cy

In [94]:
class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CustomLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.cell_list = nn.ModuleList([CustomLSTMCell(input_size, hidden_size)])
        self.cell_list.extend([CustomLSTMCell(hidden_size, hidden_size) for _ in range(1, num_layers)])
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        hidden = [torch.zeros(batch_size, self.hidden_size).to(x.device) for _ in range(self.num_layers)]
        cell = [torch.zeros(batch_size, self.hidden_size).to(x.device) for _ in range(self.num_layers)]

        for t in range(seq_len):
            for l in range(self.num_layers):
                if l == 0:
                    hidden[l], cell[l] = self.cell_list[l](x[:, t, :], (hidden[l], cell[l]))
                else:
                    hidden[l], cell[l] = self.cell_list[l](hidden[l-1], (hidden[l], cell[l]))

        out = self.fc(hidden[-1])
        return out

In [100]:
# Define model parameters
input_size = train_data.shape[1]  # Number of features
hidden_size = 64
output_size = train_labels.shape[1]  # Number of output features
num_layers = 2
model = LSTMModel(input_size, hidden_size,output_size, num_layers).to(device)
criterion=nn.MSELoss()

In [101]:
def train(model, train_dataloader, device, optimizer, criterion):
    model.train()
    total_loss = 0.0  # Initialize total loss to 0


    for i, (inputs, target) in enumerate(train_dataloader):
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        total_loss += loss.item() * inputs.size(0)

    # Return the average loss over all batches
    
    return total_loss/len(train_dataloader.dataset)


In [102]:
@torch.no_grad()
def evaluate(model, test_dataloader, device, criterion):
    model.eval()
    total_loss = 0.0  # Initialize total loss


    for i, (inputs, target) in enumerate(test_dataloader):  # Use `test_dataloader`
        inputs, target = inputs.to(device), target.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, target)

        # Accumulate the loss
        total_loss += loss.item() * inputs.size(0)

    # Return the average loss over all batches
    
    return total_loss/len(test_dataloader.dataset)


In [103]:

def run(model, train_dataloader, test_dataloader, device, epoch):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(epoch):
        loss = train(model, train_dataloader, device, optimizer,criterion)
        loss = evaluate(model, train_dataloader, device,criterion)
        test_loss = evaluate(model, test_dataloader, device,criterion)
        print(epoch, loss, test_loss)
        if (epoch + 1) == 100 or (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1:04d} | loss: {loss:.4f} '
                f'test_loss: {test_loss:.4f} ')
    
        if loss < 1e-3 and test_loss < 1e-3:
            break

In [104]:
epoch=100
run(model, train_dataloader, test_dataloader, device, epoch)

0 297348594.0498357 62367803.596072696
1 294962148.7816478 61246133.306309596
2 292617629.2658256 60157069.16599661


KeyboardInterrupt: 

In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CustomLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Combined weights for all gates
        self.weight_ih = nn.Parameter(torch.Tensor(4 * hidden_size, input_size))
        self.weight_hh = nn.Parameter(torch.Tensor(4 * hidden_size, hidden_size))
        self.bias = nn.Parameter(torch.Tensor(4 * hidden_size))
        
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / torch.sqrt(torch.tensor(self.hidden_size, dtype=torch.float))
        for weight in self.parameters():
            nn.init.uniform_(weight, -stdv, stdv)

    def forward(self, input, state):
        hx, cx = state
        gates = F.linear(input, self.weight_ih, self.bias) + F.linear(hx, self.weight_hh)

        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = F.relu(cellgate)  # Use ReLU here
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * F.relu(cy)  # Use ReLU here as well

        return hy, cy

class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CustomLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.cell_list = nn.ModuleList([CustomLSTMCell(input_size, hidden_size)])
        self.cell_list.extend([CustomLSTMCell(hidden_size, hidden_size) for _ in range(1, num_layers)])
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        hidden = [torch.zeros(batch_size, self.hidden_size).to(x.device) for _ in range(self.num_layers)]
        cell = [torch.zeros(batch_size, self.hidden_size).to(x.device) for _ in range(self.num_layers)]

        for t in range(seq_len):
            for l in range(self.num_layers):
                if l == 0:
                    hidden[l], cell[l] = self.cell_list[l](x[:, t, :], (hidden[l], cell[l]))
                else:
                    hidden[l], cell[l] = self.cell_list[l](hidden[l-1], (hidden[l], cell[l]))

        out = self.fc(hidden[-1])
        return out

# Example usage
input_size = 10
hidden_size = 20
num_layers = 2
output_size = 1
seq_length = 5
batch_size = 3

# Create an instance of the model
model = CustomLSTM(input_size, hidden_size, num_layers, output_size)

# Create a random input tensor
x = torch.randn(batch_size, seq_length, input_size)

# Forward pass
output = model(x)
print(output.shape) 

torch.Size([3, 1])
